# Extract & Transform City Bike Data

#### Source:  Ny Citi Bike Trip Histories from https://www.citibikenyc.com/system-data

##### Files Extracted:

|  File Name | URL  |
|---|---|
| 202001-citibike-tripdata.csv.zip  |  https://s3.amazonaws.com/tripdata/202001-citibike-tripdata.csv.zip |
|   |   |
|   |   |

In [1]:
import pandas as pd
from pygeocoder import Geocoder
import numpy as np
from config import google_api
#

In [2]:
#google_api

## A. Process Jan 2020 bike file and add values

### A-1.  Read the Bike File(s)

In [3]:
bike_file = "citi-bike-data/202001-citibike-tripdata.csv"
bike_df = pd.read_csv(bike_file)
bike_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1


In [4]:
bike_df.shape

(1240596, 15)

### A-2. Convert Gender to Male, Female or Unknown

In [5]:
bike_df['gender'] = bike_df['gender'].map({1:'male', 2:'female', 0:'unknown'})

bike_df['gender'].value_counts()

male       904522
female     279106
unknown     56968
Name: gender, dtype: int64

###  A-3. Detect Round Trips

In [6]:
# # https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o
# def roundtrip(row):
#     return (row['start station id'] == row['end station id'])

# bike_df.apply(lambda row: roundtrip(row), axis=1)

In [7]:
# https://stackoverflow.com/questions/48369929/creating-a-new-column-based-on-the-values-of-other-columns

bike_df.loc[(bike_df['start station id'] == bike_df['end station id']), 'Round_Trip'] = 'Y'

bike_df['Round_Trip'] = bike_df['Round_Trip'].fillna('N')

In [8]:
bike_df['Round_Trip'].value_counts()

N    1221272
Y      19324
Name: Round_Trip, dtype: int64

### A-4. Calculate Age 

In [9]:
# bike_df['birth year'].value_counts()

In [10]:
bike_df['Rider Age'] = 2020 - bike_df['birth year']
# bike_df['Rider Age'].value_counts()

In [11]:
bike_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Round_Trip,Rider Age
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,male,N,28
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,male,N,51
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,male,N,57
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,male,N,40
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,male,N,38


### A- 5. Cut Age Bins

instead of the code below, will use tableau to cut the bins and display histograms

In [12]:
# bike_df['age_bins'] = pd.cut(x=bike_df['Rider Age'], bins=[1, 12, 19, 29, 39, 49, 59, 69, 79, 89, 99, 199], 
#                              labels=['youth', 'teens', '20s', '30s', '40s', '50s', '60s', '70s', '80s', '90s', '>=100'],
#                              include_lowest=True, precision=0, right=True)

In [13]:
# bike_df['age_bins'].value_counts()

In [14]:
# big_df = pd.DataFrame()

### A-6. Cut trip duration bins
instead of coding the cut in Python, use tableu to cut bins for trip duration

## B.  Station Table

### B-1.  Create a table of station id's and names from start and end stations

#### extract a start stations df

In [15]:
station1_df = bike_df[['start station id', 'start station name', 'start station latitude',  'start station longitude']]
station1_df.columns = ['station id', 'station name', 'latitude',  'longitude']
station1_df.head()

,station id,station name,latitude,longitude
0,3496,1 Ave & E 110 St,40.792327,-73.9383
1,3496,1 Ave & E 110 St,40.792327,-73.9383
2,3496,1 Ave & E 110 St,40.792327,-73.9383
3,3496,1 Ave & E 110 St,40.792327,-73.9383
4,3496,1 Ave & E 110 St,40.792327,-73.9383


In [16]:
station1_df.shape

(1048575, 4)

#### extract a end stations df

In [17]:
station2_df = bike_df[['end station id', 'end station name', 'end station latitude', 'end station longitude']]
station2_df.columns = ['station id', 'station name', 'latitude',  'longitude']
station2_df.head()

,station id,station name,latitude,longitude
0,3345,Madison Ave & E 99 St,40.789485,-73.952429
1,3628,Lenox Ave & W 117 St,40.802557,-73.949078
2,3493,E 118 St & 3 Ave,40.799139,-73.938915
3,3345,Madison Ave & E 99 St,40.789485,-73.952429
4,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449


In [18]:
station2_df.shape

(1048575, 4)

In [65]:
# station1_df['station id'].value_counts()

#### Concat start and end station df's to form 1 list

In [20]:
station_df = pd.concat([station1_df, station2_df], ignore_index=True)

In [21]:
station_df.head()

,station id,station name,latitude,longitude
0,3496,1 Ave & E 110 St,40.792327,-73.9383
1,3496,1 Ave & E 110 St,40.792327,-73.9383
2,3496,1 Ave & E 110 St,40.792327,-73.9383
3,3496,1 Ave & E 110 St,40.792327,-73.9383
4,3496,1 Ave & E 110 St,40.792327,-73.9383


In [22]:
station_df.shape

(2097150, 4)

#### need to reduce to 1 record per station

In [23]:
station_df['station id'].value_counts()

519     17520
3255    13432
435     13302
497     13156
402     12974
        ...  
3867       13
3912        4
3245        2
3638        2
3432        1
Name: station id, Length: 894, dtype: int64

In [24]:
#### check a value (there many hits)
station_df[station_df['station id'] == 519]

,station id,station name,latitude,longitude
741207,519,Pershing Square North,40.751873,-73.977706
741208,519,Pershing Square North,40.751873,-73.977706
741209,519,Pershing Square North,40.751873,-73.977706
741210,519,Pershing Square North,40.751873,-73.977706
741211,519,Pershing Square North,40.751873,-73.977706
...,...,...,...,...
2084174,519,Pershing Square North,40.751873,-73.977706
2085177,519,Pershing Square North,40.751873,-73.977706
2087331,519,Pershing Square North,40.751873,-73.977706
2087438,519,Pershing Square North,40.751873,-73.977706


#### drop duplictes

In [60]:
# drop duplicates 
station_df.drop_duplicates(subset=['station id', 'station name'], keep='first', inplace=True)
station_df.reset_index(drop=True, inplace=True)
station_df['station id'].value_counts()

2022    1
3071    1
3085    1
3083    1
3082    1
       ..
3531    1
3530    1
3529    1
3528    1
72      1
Name: station id, Length: 894, dtype: int64

In [26]:
station_df.shape

(894, 4)

In [27]:
#### check a value - only 1 value
station_df[station_df['station id'] == 519]

,station id,station name,latitude,longitude
668,519,Pershing Square North,40.751873,-73.977706


In [59]:
#station_df.tail()

### B-2.  Add zip code and other loction data to the station data using reverse geocoder and lat/long data

In [42]:
def reverseGeo(lat, lon) :
    
    results = Geocoder(google_api).reverse_geocode(lat, lon)
#     geoDict = {"zipcode" : results.postal_code, "burrough" : results.administrative_area_level_2}
    
    return(results.postal_code, results.sublocality, results.city, results.administrative_area_level_2, results.administrative_area_level_1)

In [50]:
station_df[["zipcode", "burrough", "city", "county", "state"]] = station_df.apply(lambda x: pd.Series(reverseGeo(x.latitude, x.longitude)), axis=1)

In [51]:
station_df

,station id,station name,latitude,longitude,zipcode,burrough,city,county,state
0,3496,1 Ave & E 110 St,40.792327,-73.938300,10029,Manhattan,New York,New York County,New York
1,504,1 Ave & E 16 St,40.732219,-73.981656,10009,Manhattan,New York,New York County,New York
2,2003,1 Ave & E 18 St,40.733812,-73.980544,10009,Manhattan,New York,New York County,New York
3,536,1 Ave & E 30 St,40.741444,-73.975361,10016,Manhattan,New York,New York County,New York
4,455,1 Ave & E 44 St,40.750020,-73.969053,10017,Manhattan,New York,New York County,New York
...,...,...,...,...,...,...,...,...,...
889,3911,Wyckoff St & Nevins St,40.683426,-73.984275,11217,Brooklyn,None,Kings County,New York
890,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,11249,Brooklyn,None,Kings County,New York
891,3638,Washington St,40.724294,-74.035483,07310,None,Jersey City,Hudson County,New Jersey
892,3245,NYCBS DEPOT - DELANCEY,40.716444,-73.982331,10002,Manhattan,New York,New York County,New York


### experiment with googl geocder api

#### test the geocoder reverse_gecode API

In [ ]:
# lat = bike_df['start station latitude'][0]
# lon = bike_df['start station longitude'][0]

# print(f"Lat: {lat}, Lon: {lon}")

In [ ]:
# results = Geocoder(google_api).reverse_geocode(lat, lon)

In [ ]:
# results

In [ ]:
# results.raw

In [ ]:
# results.formatted_address

In [ ]:
# results.city

In [ ]:
# burrough
# results.sublocality

In [ ]:
# results.postal_code

In [ ]:
# results.county

In [ ]:
# county
# results.administrative_area_level_2

In [ ]:
# state
# results.administrative_area_level_1

In [ ]:
# results.country

In [66]:
# row_df = station_df.loc[0 : 0]
# row_df

In [32]:
# geoSeries = row_df.apply(lambda x: pd.Series(reverseGeo(x.latitude, x.longitude)), axis=1)

In [44]:
# geoSeries

In [45]:
# row_df[["zipcode", "burrough", "city", "county", "state"]] = row_df.apply(lambda x: pd.Series(reverseGeo(x.latitude, x.longitude)), axis=1)

In [67]:
# row_df

### B-3 Group By Station Id's and Count StartStations and End Stations

decided to do the counts in Tableau

In [ ]:
# byStartStationGB = bike_df.groupby('start station id')
# byEndStationGB = bike_df.groupby('end station id')

# startCnts = byStartStationGB.count()['start station name']
# endCnts = byEndStationGB.count()['end station name']

In [ ]:
# cnt_df = pd.DataFrame({"startStationCount" : startCnts, "endStationCount" : endCnts }) 
# cnt_df

### Write out tranformed data files to read into Tableau

In [16]:
bike_filepath = "xformed_data/bike_data_2020-01.csv"
bike_df.to_csv(bike_filepath, header=True, index=False) 

In [56]:
station_filepath = "xformed_data/station_data.csv"
station_df.to_csv(station_filepath, header=True, index=False)

In [15]:
bike_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Round_Trip,Rider Age
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,male,N,28
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,male,N,51
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,male,N,57
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,male,N,40
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,male,N,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240591,1587,2020-01-31 23:59:26.8820,2020-02-01 00:25:54.4890,3244,University Pl & E 8 St,40.731437,-73.994903,3092,Berry St & N 8 St,40.719009,-73.958525,40662,Subscriber,1990,male,N,30
1240592,222,2020-01-31 23:59:32.6410,2020-02-01 00:03:15.4720,383,Greenwich Ave & Charles St,40.735238,-74.000271,383,Greenwich Ave & Charles St,40.735238,-74.000271,28722,Subscriber,1983,male,Y,37
1240593,163,2020-01-31 23:59:39.1780,2020-02-01 00:02:23.0400,150,E 2 St & Avenue C,40.720874,-73.980858,411,E 6 St & Avenue D,40.722281,-73.976687,32530,Subscriber,1958,female,N,62
1240594,327,2020-01-31 23:59:49.2310,2020-02-01 00:05:16.3790,483,E 12 St & 3 Ave,40.732233,-73.988900,3718,E 11 St & Avenue B,40.727464,-73.979504,15314,Customer,1994,male,N,26
